# 1. proyecto GeoSpacial Data

In [202]:
from pymongo import MongoClient
import pandas as pd
import time
import importlib
import data.cleaning as clean
importlib.reload(clean)

<module 'data.cleaning' from 'D:\\ironhack\\proyectos\\GeoSpatialData_proy3\\data\\cleaning.py'>

Todo el proceso para descargar la base de datos Empresas de Mongo.

In [4]:
client = MongoClient("localhost:27017")

In [5]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [6]:
client.list_database_names()

['Ironhack', 'admin', 'config', 'local']

In [12]:
db = client["Ironhack"]

In [13]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack')

In [15]:
c = db.get_collection("companies")

Explore the data base to get the information I want for the company.

In [16]:
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [21]:
list(c.find({}, {"category_code":1}))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'), 'category_code': 'web'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8b'), 'category_code': 'enterprise'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'), 'category_code': 'software'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'), 'category_code': 'news'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'), 'category_code': 'social'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
  'category_code': 'network_hosting'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d90'), 'category_code': 'web'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d91'), 'category_code': 'web'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d92'), 'category_code': 'games_video'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d93'), 'category_code': 'web'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d94'), 'category_code': 'social'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d95'), 'category_code': 'web'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d96'), 'category_code': 'web'},
 {'_id': ObjectId('52cd

#### Escogueré the music category for this proyect (my business)
Getting the music buissnes companies and start analizing them.

In [24]:
list(c.find({"category_code":"music"}, {"name":1, "_id":0, "category_code":1}))

[{'name': 'Slacker', 'category_code': 'music'},
 {'name': 'Thumbplay', 'category_code': 'music'},
 {'name': 'imeem', 'category_code': 'music'},
 {'name': 'Animoto', 'category_code': 'music'},
 {'name': 'Myxer', 'category_code': 'music'},
 {'name': 'Songkick', 'category_code': 'music'},
 {'name': 'OurStage', 'category_code': 'music'},
 {'name': 'Songza', 'category_code': 'music'},
 {'name': 'Deezer', 'category_code': 'music'},
 {'name': 'blinkbox music', 'category_code': 'music'},
 {'name': 'MOG', 'category_code': 'music'},
 {'name': '7digital', 'category_code': 'music'},
 {'name': 'The Echo Nest', 'category_code': 'music'},
 {'name': 'TuneWiki', 'category_code': 'music'},
 {'name': 'Spotify', 'category_code': 'music'},
 {'name': 'TuneUp', 'category_code': 'music'},
 {'name': 'MIKESTAR', 'category_code': 'music'},
 {'name': 'Nimbit', 'category_code': 'music'},
 {'name': 'Bandsintown', 'category_code': 'music'},
 {'name': 'Rhapsody', 'category_code': 'music'},
 {'name': 'Band Metrics', '

Get the companies Ubication COUNTRY.

In [290]:
def music_countries (category_code, country_code):
    condition1 = {"category_code":category_code}
    projection = {"name":1, "_id":0, "offices.country_code":country_code}
    countries = (list(c.find(condition1, 
               projection)))
    
    listcountry = [countries[i]["offices"][j]["country_code"] for i in range(len(countries)) for j in range(len(countries[i]["offices"]))]
    return listcountry
    

In [291]:
countries = music_countries ("music", 1)

In [292]:
countries[0]
countries[0]["offices"]
countries[0]["offices"][0]["country_code"]

TypeError: string indices must be integers

In [293]:
# change it to a dictionary
country = {"country": listcountry}

In [294]:
# make it a data frame
dfcountry = pd.DataFrame(country)

In [184]:
dfcountry.value_counts()[:10]

country
USA        30
GBR         5
FRA         4
DEU         2
BEL         1
BRA         1
ESP         1
ITA         1
LUX         1
NLD         1
dtype: int64

In [185]:
exportar data frame a csv para abrirlo en el archivo de tablas.

SyntaxError: invalid syntax (1928548442.py, line 1)

hacer otra columna del value count de los paises

In [187]:
dfcountry["count"] = dfcountry.groupby(["country"]).transform("count")

ValueError: Cannot set a DataFrame with multiple columns to the single column count

In [ ]:
dfcountry

We see that for much USA is the place for a music company, I wanted to see if I could go to another country that had some type of competition between the USA, but if the company wants to grow, need to be in the USA. 

### Searching for the city

In [295]:
def music_cities (category_code, cities):
    condition1 = {"category_code":category_code}
    projection = {"name":1, "_id":0, "offices.city":cities}
    city = (list(c.find(condition1, 
               projection)))
    
    lstcity = [city[i]["offices"][j]["city"] for i in range(len(city)) for j in range(len(city[i]["offices"]))]
    return lstcity

In [296]:
lstcity = music_cities ("music", 1)

In [307]:
def combine_lists (lst1, lst2):
    # change the 2 lists into a dictionary
    location = {"cities": lst1,
            "country": lst2} 
    # create them a Data Frame.
    location = pd.DataFrame(location)
    # create a new column with the value_counts of cities.
    location["count"] = location.groupby(["cities"])["country"].transform("count")
    return location

In [314]:
location_df = combine_lists (lstcity, listcountry)

In [322]:
location_df

,cities,country,count
0,San Diego,USA,2
1,New York,USA,6
2,San Francisco,USA,9
3,New York,USA,6
4,San Francisco,USA,9
5,Austin,USA,1
6,Deerfield Beach,USA,1
7,London,GBR,4
8,Chelmsford,USA,1
9,Long Island City,USA,1


Will do a drop duplicated and drop na 

In [318]:
def clean_df_citycountry (df):
    df = df.combine_first(df).reset_index().reindex(location.columns, axis=1).drop_duplicates()
    df.dropna(how="any", inplace=True)
    return df

In [319]:
location_clean = clean_df_citycountry (location_df)

Export the data frame of location into a CSV file.

In [346]:
def export_location (df):
    return df.to_csv(f"D:\\ironhack\\proyectos\\GeoSpatialData_proy3\csv\\location_clean.csv")
    

In [348]:
export_csv (location_clean)